# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-03-03 05:07:28] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=35605, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=314531524, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metr

[2025-03-03 05:07:48 TP0] Init torch distributed begin.
[2025-03-03 05:07:49 TP0] Load weight begin. avail mem=62.69 GB


[2025-03-03 05:07:49 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-03 05:07:49 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.30it/s]


...

[2025-03-03 05:07:53 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-03-03 05:07:54] INFO:     Started server process [3015965]
[2025-03-03 05:07:54] INFO:     Waiting for application startup.
[2025-03-03 05:07:54] INFO:     Application startup complete.
[2025-03-03 05:07:54] INFO:     Uvicorn running on http://0.0.0.0:35605 (Press CTRL+C to quit)


[2025-03-03 05:07:54] INFO:     127.0.0.1:56364 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-03 05:07:54] INFO:     127.0.0.1:56378 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-03 05:07:54 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


Server started on http://localhost:35605


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-03-03 05:07:59] INFO:     127.0.0.1:52418 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-03-03 05:08:00 TP0] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-03-03 05:08:00] INFO:     127.0.0.1:56392 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 05:08:00] The server is fired up and ready to roll!


[2025-03-03 05:08:01] INFO:     127.0.0.1:52430 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-03-03 05:08:01 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, cache hit rate: 24.66%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 05:08:01 TP0] Decode batch. #running-req: 1, #token: 48, token usage: 0.00, gen throughput (token/s): 6.30, #queue-req: 0


[2025-03-03 05:08:01] INFO:     127.0.0.1:52434 - "POST /generate HTTP/1.1" 200 OK


[2025-03-03 05:08:01 TP0] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 69, cache hit rate: 52.73%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 05:08:01 TP0] Decode batch. #running-req: 1, #token: 110, token usage: 0.01, gen throughput (token/s): 57.68, #queue-req: 0


[2025-03-03 05:08:02] INFO:     127.0.0.1:52436 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-03-03 05:08:02 TP0] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, cache hit rate: 47.98%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-03-03 05:08:02] INFO:     127.0.0.1:52444 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 05:08:02 TP0] Prefill batch. #new-seq: 2, #new-token: 5, #cached-token: 62, cache hit rate: 59.25%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-03-03 05:08:02] INFO:     127.0.0.1:52460 - "POST /generate HTTP/1.1" 200 OK


[2025-03-03 05:08:02 TP0] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, cache hit rate: 61.09%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 05:08:02 TP0] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, gen throughput (token/s): 64.87, #queue-req: 0


[2025-03-03 05:08:02] INFO:     127.0.0.1:52476 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-03-03 05:08:02 TP0] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, cache hit rate: 56.67%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-03-03 05:08:02 TP0] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, cache hit rate: 53.30%, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-03-03 05:08:03 TP0] Decode batch. #running-req: 2, #token: 92, token usage: 0.00, gen throughput (token/s): 83.98, #queue-req: 0


[2025-03-03 05:08:03 TP0] Decode batch. #running-req: 2, #token: 172, token usage: 0.01, gen throughput (token/s): 118.38, #queue-req: 0


[2025-03-03 05:08:04 TP0] Decode batch. #running-req: 2, #token: 252, token usage: 0.01, gen throughput (token/s): 115.97, #queue-req: 0


[2025-03-03 05:08:05] INFO:     127.0.0.1:52478 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 05:08:05 TP0] Decode batch. #running-req: 1, #token: 188, token usage: 0.01, gen throughput (token/s): 107.08, #queue-req: 0


[2025-03-03 05:08:06 TP0] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, gen throughput (token/s): 61.28, #queue-req: 0
[2025-03-03 05:08:06] INFO:     127.0.0.1:52490 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 05:08:06 TP0] Prefill batch. #new-seq: 1, #new-token: 336, #cached-token: 39, cache hit rate: 32.78%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 05:08:06 TP0] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, gen throughput (token/s): 57.79, #queue-req: 0


[2025-03-03 05:08:07 TP0] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, gen throughput (token/s): 60.13, #queue-req: 0


[2025-03-03 05:08:08 TP0] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, gen throughput (token/s): 61.57, #queue-req: 0
[2025-03-03 05:08:08] INFO:     127.0.0.1:52496 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-03-03 05:08:08 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, cache hit rate: 33.05%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 05:08:09] INFO:     127.0.0.1:57014 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-03-03 05:08:10 TP0] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, cache hit rate: 33.22%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 05:08:10 TP0] Decode batch. #running-req: 1, #token: 51, token usage: 0.00, gen throughput (token/s): 15.18, #queue-req: 0


[2025-03-03 05:08:11 TP0] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, gen throughput (token/s): 57.08, #queue-req: 0


[2025-03-03 05:08:12 TP0] Decode batch. #running-req: 1, #token: 131, token usage: 0.01, gen throughput (token/s): 60.51, #queue-req: 0


[2025-03-03 05:08:12] INFO:     127.0.0.1:57026 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-03-03 05:08:12 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, cache hit rate: 34.18%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-03-03 05:08:12] INFO:     127.0.0.1:57028 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

[2025-03-03 05:08:12 TP0] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, cache hit rate: 35.01%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-03-03 05:08:12 TP0] Prefill batch. #new-seq: 2, #new-token: 18, #cached-token: 34, cache hit rate: 36.68%, token usage: 0.00, #running-req: 1, #queue-req: 0


 33%|███▎      | 1/3 [00:00<00:00,  5.46it/s]

[2025-03-03 05:08:12] INFO:     127.0.0.1:57042 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 05:08:12] INFO:     127.0.0.1:57044 - "POST /generate HTTP/1.1" 200 OK


100%|██████████| 3/3 [00:00<00:00, 14.01it/s]

[2025-03-03 05:08:12] INFO:     127.0.0.1:57030 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-03-03 05:08:12] INFO:     127.0.0.1:57054 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 05:08:12 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, cache hit rate: 38.27%, token usage: 0.00, #running-req: 0, #queue-req: 0
The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-03-03 05:08:12 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, cache hit rate: 38.48%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 05:08:12] INFO:     127.0.0.1:57058 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/references/supported_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-03-03 05:08:27] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=39119, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=247021470, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, en

[2025-03-03 05:08:46 TP0] Overlap scheduler is disabled for multimodal models.
[2025-03-03 05:08:46 TP0] Automatically reduce --mem-fraction-static to 0.836 because this is a multimodal model.
[2025-03-03 05:08:46 TP0] Init torch distributed begin.


[2025-03-03 05:08:47 TP0] Load weight begin. avail mem=78.81 GB
[2025-03-03 05:08:47 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-03 05:09:05 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:02,  1.60it/s]


...

[2025-03-03 05:09:10 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=128000
[2025-03-03 05:09:10] INFO:     Started server process [3018778]
[2025-03-03 05:09:10] INFO:     Waiting for application startup.
[2025-03-03 05:09:10] INFO:     Application startup complete.
[2025-03-03 05:09:10] INFO:     Uvicorn running on http://0.0.0.0:39119 (Press CTRL+C to quit)


[2025-03-03 05:09:11] INFO:     127.0.0.1:43760 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-03 05:09:11] INFO:     127.0.0.1:43770 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-03 05:09:11 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 05:09:13] INFO:     127.0.0.1:43786 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 05:09:13] The server is fired up and ready to roll!


Server started on http://localhost:39119


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-03-03 05:09:16] INFO:     127.0.0.1:43788 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-03-03 05:09:25 TP0] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 05:09:29 TP0] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, gen throughput (token/s): 2.09, #queue-req: 0


[2025-03-03 05:09:30 TP0] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, gen throughput (token/s): 58.50, #queue-req: 0


[2025-03-03 05:09:30] INFO:     127.0.0.1:52680 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)